# Catalog Cleanup for Informatica Migration

First, we import the necessary libraries for this work.
***Note***: a **.env** file is needed wherever this notebook is stored, which will have the Environment Variables:
- CP4D_PROD_URL: The url for the PROD CP4D instance, and it must contain 'https://' prefixed.
- CP4D_USERNAME: Your MSK ID name.
- USERPASS: Your MSK ID Password.

In [6]:
import pandas as pd
import glob
from io import StringIO
import numpy as np
import re
import os
import pymssql
import pyodbc
import requests
import xlsxwriter
import ssl
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
from dotenv import load_dotenv
load_dotenv('.env')

True

In [7]:
pd.set_option('display.max_rows',150000)
pd.set_option('display.max_columns',100)
pd.set_option('display.width', 10000)

Below, we do an API call to the CP4D instance to retrieve all the Concepts and their metadata, such as definition, approval data, authoritative sources.
This will be saved in an Excel spreadsheet for documentation and further manipulation.

In [8]:
# url = os.getenv("CP4D_PROD_URL")
# # url = os.getenv("CP4D_QA_URL")
# user = os.getenv("CP4D_USERNAME")
# pwd = os.getenv("USERPASS")
# pd.set_option('display.max_rows',150000)
# pd.set_option('display.max_columns',10)
# pd.set_option('display.width', 10000)
#
# # logging.basicConfig(filename='CPAPI.log',
# #                     filemode='a',
# #                     format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
# #                     datefmt='%H:%M:%S',
# #                     level=logging.INFO)
# body = {
#   "password": pwd,
#   "username": user,
#   "useLdapGroup": False,
# }
# auth = requests.post(f"{url}/icp4d-api/v1/authorize", json=body)
# # print(auth.text)
# # print(json.dumps(auth, indent=3))
# headers = {"Authorization": f"Bearer {auth.json()['token']}"}
# paths={'projects':'/v2/projects',
#        'projects_v1':'/ibm/iis/dq/da/rest/v1/workspaces',
#        'catalogs':'/v2/catalogs',
#        'assets': '/v2/assets',
#        'asset_types':'/v2/asset_types',
#        'asset_files':'/v2/asset_files',
#        'data_assets': '/v2/asset_types/data_asset/search',
#        'data_class': '/v3/data_classes',
#        'dc_search': '/v3/data_classes/relationships/search',
#        'folder_assets': '/v2/folder_assets',
#        'terms':'/v3/glossary_terms',
#        'export_terms':'/v3/governance_artifact_types/glossary_term/export',
#        'rules':'/v3/rules/relationships/search',
#        'reference':'/v3/reference_data',
#        'cnxs':'/v2/connections',
#        'jobs':'/v2/jobs'}
#
# btdl = requests.get(f"{url}{paths['export_terms']}", headers=headers).text
# writer = pd.ExcelWriter('WKC-Extract-PROD.xlsx', engine='xlsxwriter')
# btdl = pd.read_csv(StringIO(btdl),sep=',').to_excel(writer,index='False')
# writer.save()

We then load the Concepts for the comparison work via a Pandas dataframe.

In [9]:
wkc = pd.read_excel('WKC-Extract-PROD.xlsx')
wkc = wkc[['Name', 'Description','Stewards', 'Business Start', 'Data Classes','custom_00. Date Approved by Stewardship Council', 'custom_03. Authoritative Source System', 'custom_04. Authoritative Source Table', 'custom_05. Authoritative Source Column']]
wkc.columns = ['Name', 'Description','Stewards', 'Business Start', 'Data Classes','Stewardship Council Approval Date', 'Authoritative Source System', 'Authoritative Source Table', 'Authoritative Source Column']


# print(iic2)

The stewardship working templates should be saved, preferably in a folder, where we can read through them and capture the necessary tabs that contain the concepts and all the stewardship work done for them for each of our domains.
A cleanup depending on the domain will be done to make sure it aligns with the WKC information for comparison.
The ***udf*** variable used below will be our *Master* list of concepts and definitions we will be changing.

In [10]:
files = glob.glob('./Stewardship/*')
udf = pd.DataFrame(columns=['System','Table Name','Column','Vendor Definition','Enterprise Concept','Working Definition','Authoritative?','UDF Ingest?','PHI Indicator','Data Class','Squad','Status'])
for file in files:
  df = pd.ExcelFile(file)
  sheets = df.sheet_names
  # print(file.replace('./Stewardship/',''), df.sheet_names)
  for sheet in sheets:
    if 'CCDE' in file:
      pass
    else:
      print(f"Processing {sheet} on file {file.replace('./Stewardship/','')}")
      f = df.parse(sheet)
      if 'Working Definition' in f.columns:
        if 'Tag' in f.columns:
          pass
        else:
          f['Tag'] = np.NaN
        if 'Telemedicine' in file:
          f = f.drop(['Source Table (Telemed DB)','Source Column'], axis=1)
          f.columns = ['ID', 'Squad', 'Table Name', 'Column', '% null', '% fill', 'Vendor Definition', 'Manually tagged in WKC?', 'Enterprise Concept', 'Working Definition', 'Related Concept', 'Allowable Values \n(if applicable) ', 'Authoritative?', 'UDF Ingest?', 'UDF Environments', 'PHI Indicator', 'Data Class', 'Tag', 'PHI DType', 'Masking Required/ Supported DType? (Y/N)', 'Notes from Stewardship', 'Notes from Data Stewards', 'Scope Review Notes', 'Questions / Follow-up with Stewards', 'Status', 'Data Class Status', 'Date PHI Class  Signed off (Stewardship Team)', 'Date Class Tagged in WKC', 'Catalog Masking Observed (4.05)', 'DV/Dbeaver Masking Observed (4.05) ', 'Testing account', 'Notes', 'Address  Line 3']
        f['System'] = file.replace('./Stewardship/','').replace('.xlsm','').replace('Stewardship-','').replace('.xlsx','')
        newdf = f[['System','Table Name','Column','Vendor Definition','Enterprise Concept','Working Definition','Authoritative?','UDF Ingest?','PHI Indicator','Data Class','Tag','Squad', 'Notes from Stewardship', 'Notes from Data Stewards', 'Scope Review Notes', 'Questions / Follow-up with Stewards','Status']]
        udf['Table Name'] = udf['Table Name'].str.lower().replace(' ','')
        udf['Column'] = udf['Column'].str.strip()
        udf['Column'] = udf['Column'].str.lower()
        udf = pd.concat([udf,newdf], axis=0, ignore_index=True)
        # print(udf.shape[0])
      else:
        pass

for col in udf.columns:
  udf[col] = udf[col].str.strip()
udf = udf.fillna('')

Processing Tracking & Resources on file Stewardship-CIS.xlsm
Processing Global Checklist on file Stewardship-CIS.xlsm
Processing ClinDoc-MVP on file Stewardship-CIS.xlsm
Processing Meds-OrderedMVPColumns on file Stewardship-CIS.xlsm
Processing Meds-AdminColumns on file Stewardship-CIS.xlsm
Processing Meds-OrderNonMVPColumns on file Stewardship-CIS.xlsm
Processing Lists on file Stewardship-CIS.xlsm
Processing Guidelines & Checklist on file Stewardship-Telemedicine.xlsx
Processing Authoritative Source Tables on file Stewardship-Telemedicine.xlsx
Processing Microsoft V2 - PROD Columns on file Stewardship-Telemedicine.xlsx
Processing Guidelines & Checklist on file Stewardship-Pathology.xlsm
Processing Global Checklist on file Stewardship-Pathology.xlsm
Processing Team List on file Stewardship-Pathology.xlsm
Processing Tracking & Resources on file Stewardship-Pathology.xlsm
Processing Stewardship Tracking - DD on file Stewardship-Pathology.xlsm
Processing Stewardship Tracking - non-MVP on f

In [11]:
iic = pd.DataFrame(udf[['System','Table Name','Column','Enterprise Concept']][(udf['Enterprise Concept'].str.contains('Identifier')) | (udf['Enterprise Concept'].str.contains('Internal Identifier'))]).reset_index(drop=True)#.drop('index',axis=1)
# iic['Enterprise Concept'] = iic['Enterprise Concept'].str.replace(r' \n',' ', regex=True).replace(r'\n',' ', regex=True)
iic[['root','suffix','a']] = iic['Enterprise Concept'].str.split('Internal', expand=True)
iic[['root','suffix1','a']] = iic['root'].str.split('Identifier',expand= True)
iic['suffix'] = iic['suffix'].str.replace('Identifier','Internal Identifier').fillna('Identifier')
iic = iic.drop(['suffix1','a'],axis=1).sort_values(['Enterprise Concept','suffix']).drop_duplicates()
iic1 = iic[iic.duplicated('root',keep=False)].reset_index(drop=True)
iic1 = iic1.drop_duplicates(subset=['root','suffix'])
iic1['suffix'] = iic1['suffix'].str.strip()
print(iic1.shape)

iic2 = iic.drop_duplicates(subset=['System','Table Name','Column','root'],keep='last')
iic2 = iic2[iic2['suffix'].str.contains('Internal Identifier')]
print(iic2[iic2['System'] == 'Pathology'].shape)


(246, 6)
(84, 6)


Next, we import the IBM export from the IGC site here: https://zen-cpd-zen.msk-udf-cpd-prod-f5abc063be4517f30ecdedb296bcfb96-0000.us-east.containers.appdomain.cloud/ibm/iis/igc/
In this platform, we will use the Queries module and the following queries to extract the needed files:
- Database Schema, Table, Column, Definitions and Concepts -> which we will save under the name 'IBM-catalog-extract'
- Database Columns and their Data Classes -> which we will save under the name 'IBM-catalog-dataclass'


In [12]:
cat = pd.read_csv('IBM-catalog-extract.csv', encoding='unicode_escape')#.drop(['Asset Name'], axis=1)
dc = pd.read_csv('IBM-catalog-dataclass.csv').drop_duplicates(subset=['Schema','Table Name','Column'],keep='last').fillna('')

cp4d = cat.merge(dc, on=['Schema','Table Name','Column'],how='left').fillna('').sort_values(['Schema','Table Name','Column'], ignore_index=True)

cp4d['Table Name'] = cp4d['Table Name'].str.replace('_V','')
cp4d['Table Name'] = cp4d['Table Name'].str.lower()
cp4d['Column'] = cp4d['Column'].str.lower()

for idx, row in cp4d.iterrows():
    if row['Schema'] == 'UDF_RMS_RAW_DV':
        row['Table Name'] = row['Table Name'].split('__')[0].replace('_','')
for col in cp4d.columns:
  cp4d[col] = cp4d[col].str.strip()

# cp4d = cp4d.drop(['asset','rule_name','rule_description','rule_action'],axis=1).fillna('')
# cp4d = cp4d[['Schema', 'Table','column_name', 'column_terms', 'column_tags', 'data_class']].fillna('')
cp4d.columns = ['Schema', 'Table Name','Column', 'WKC Concept Name', 'WKC Definition', 'Data Class']
cp4d = cp4d.fillna('')

Let's start by checking the concept names and whether they match between our Templates and WKC currently.

In [13]:
conc = udf.merge(wkc, left_on=['Enterprise Concept'], right_on=['Name'], indicator='Template or WKC')
conc = conc[['System','Enterprise Concept','Working Definition','Description','Template or WKC']]
conc.columns = ['System','Enterprise Concept','Working Definition','WKC Definition','Template or WKC']
conc['Def_Match'] = np.where(conc['Working Definition'] == conc['WKC Definition'],'Y','N')
conc = conc.drop_duplicates(subset=['Enterprise Concept','Working Definition'], keep='first')

We will create a combined IBM master file that contains important metadata, such as Authoritative Sources, Data Class, Council Approval date, etc.

In [14]:
ibm = wkc.merge(cp4d, left_on='Name', right_on='WKC Concept Name',how='left')

Then we will merge our Stewardship spreadsheet information to the combined IBM master file. This will be our guide for our cleanup.
We do this by merging the two sets based on the Table Name and column values.

In [15]:
df1 = udf.merge(ibm, on=['Table Name','Column'], suffixes=['_Template','_IBM'], how='left', indicator='Template or IBM')
df1 = df1.replace(np.NaN, '')
# print(df1.columns)

Below, we will extract the concepts where the Concept names do not match between UDF stewardship templates and CP4D extract

In [16]:
cnm = df1[['System', 'Table Name', 'Column','Enterprise Concept', 'WKC Concept Name','Template or IBM']][(df1['Enterprise Concept'] != df1['WKC Concept Name']) & (df1['Status'].isin(['Complete','Signed Off Stewardship']))].reset_index(drop=True)

Next, let's find those records where the Working Definition either does not match to that in IBM, or is blank.

In [17]:
dnm = df1[['System', 'Table Name', 'Column','Enterprise Concept','Name', 'Working Definition','Description','Template or IBM']][((df1['Working Definition'] != '') & (df1['Working Definition'] != df1['Description']) & (df1['Name'] != '')) & (df1['Status'].isin(['Complete','Signed Off Stewardship','Sign off Complete']))].reset_index(drop=True)

Then, we will extract the concepts indicated as PHI in our stewardship templates that do ***not*** have a matching **Data Class** in CP4D

In [18]:
phidc = df1[['System', 'Table Name', 'Column','Data Class_Template', 'Data Class_IBM','Tag','Template or IBM']][(df1['PHI Indicator'].isin(['y','Y','Yes','YES','PHI'])) & (df1['Data Class_Template'] != df1['Data Class_IBM'])].reset_index(drop=True)

Another check here is to identify columns that could potentially be PHI that were not so during stewardship

In [19]:
phic = pd.read_excel('./Stewardship/PHI-Categories.xlsx')
phir = pd.DataFrame()
for idx, row in phic.iterrows():
    cl = row['Regex'].split(',')
    l = '|'.join(cl)
    cm = row['Regex2'].split(',')
    m = '|'.join(cm)
    print(f"PHI Check for {row['Potential PHI']} with parameters {row['Regex']} {'---'*20}")
    phint = df1[(df1['Column'].str.contains(l, regex=True, case=False)) | (df1['Enterprise Concept'].str.contains(m, regex=True, case=False))].reset_index(drop=True)

    # phint = df1[(df1['Enterprise Concept'].str.contains(m, regex=True, case=False))].reset_index(drop=True)
    phint['Potential PHI - Column'] = np.where(phint['Column'].str.contains(l,regex=True, case=False), row['Potential PHI'],'')
    phint['Potential PHI - Concept'] = np.where(phint['Enterprise Concept'].str.contains(m,regex=True, case=False), row['Potential PHI'],'')
    # print(phint1)
    phint = phint[['System', 'Table Name', 'Column','Enterprise Concept', 'Working Definition', 'Squad', 'Potential PHI - Column','Potential PHI - Concept','Data Class_Template', 'WKC Concept Name', 'Data Class_IBM','Template or IBM']]
    phir = pd.concat([phir,phint], axis=0, ignore_index=True).drop_duplicates(subset=['Table Name','Column'],keep='last')
phir = phir[(phir['Potential PHI - Column'] != '' )| (phir['Potential PHI - Concept'] != '')]
# print(phir[['System','Column','Enterprise Concept','Working Definition','Potential PHI - Column','Potential PHI - Concept']])

PHI Check for Account with parameters acc.*num ------------------------------------------------------------
PHI Check for Address with parameters addr.*lin,addr.* ------------------------------------------------------------
PHI Check for Address with parameters postal.*,zip.* ------------------------------------------------------------
PHI Check for Address with parameters province ------------------------------------------------------------
PHI Check for Address with parameters city,town ------------------------------------------------------------
PHI Check for Address with parameters latitude,longitude,geographic,geo.*coor ------------------------------------------------------------
PHI Check for Address with parameters eircode ------------------------------------------------------------
PHI Check for Address with parameters post.*code ------------------------------------------------------------
PHI Check for Address with parameters county --------------------------------------------

Next, Let\'s check that the Data Classes currently attached to the terms in the Stewardship templates are the ones up-to-date.

In [20]:
dpr = pd.read_excel('./Stewardship/DPR.xlsx').fillna('')
dcw = pd.read_excel('WKC-Data-classes.xlsx').fillna('')
dcr = dcw.merge(dpr, on='Data Class', suffixes=['_IBM','_MSK'], indicator=True, how='left')
dcr = dcr[['Custom/OOB',	'PHI/PII',	'PHI Category',	'Parent Data Class',	'Data Class',	'Data Type',	'Max Length',	'Min Length',	'Description_IBM', 'RegEx']]

### DUPLICATING COLUMN TO HAVE SEPARATE COLUMNS ON JOIN ###
dpr['Data Class_DPR'] = dpr['Data Class']
dpr = dpr.drop('Data Class', axis=1)

dprnm = udf.merge(dpr, left_on='Data Class', right_on='Data Class_DPR', how='left', indicator='Template or IBM')
dprnm['PHI/PII'] = dprnm['PHI/PII'].fillna('')
dprnm = dprnm[['System', 'Table Name','Column','Enterprise Concept','Working Definition','Data Class','Data Class_DPR','PHI/PII','PHI Indicator']][(dprnm['Data Class'] != '') & (dprnm['Data Class'] != dprnm['Data Class_DPR'])].replace(np.NaN,'').reset_index(drop=True)


Lastly, we will create an Excel file with all our findings, and save each of the data sets we've created into their own tab in the file.

In [21]:
file = 'DC-Cleanup1.xlsx'
writer = pd.ExcelWriter(file, engine='xlsxwriter')
udf.to_excel(writer, index=False, sheet_name='Master')
conc.to_excel(writer,index=False, sheet_name='Concept-in-WKC')
cnm.to_excel(writer, index=False, sheet_name='ConceptName-Mismatch')
iic1.to_excel(writer, index=False, sheet_name='InternalIdent-vs-Ident')
iic2.to_excel(writer, index=False, sheet_name='InternalIdent-Change')
dnm.to_excel(writer, index=False, sheet_name='Definition-Mismatch')
dcr.to_excel(writer, index=False, sheet_name='PHI-DataClass')
phidc.to_excel(writer, index=False, sheet_name='PHI-DataClass-Mismatch')
phir.to_excel(writer, index=False, sheet_name='PHI-Review')
dprnm.to_excel(writer, index=False, sheet_name='DataClass-Update')
writer.save()